In [1]:
using DelimitedFiles
#using Pkg
#Pkg.add("Distributions")
#Pkg.update("Distributions")
using Distributions
using Printf

In [2]:
fname = "data/item.txt"
dataset = readdlm(fname, ' ', Int64)
dataset_origin = readdlm(fname, ' ', Int64)

662×118 Array{Int64,2}:
 1  0  1  1    1  1  1  1  1  1  1  1  …  1  1    0  0  1  1  0  1  1  0  1
 1  0  1  1    1  0  0  1  1  0  0  0     0  1    1  0  1  1  1  1  0  0  1
 0  1  1  1    1  1  0  1  0  1  1  1     1  1    0  0  1  1  1  1  0  1  1
 1  0  1  0    1  1  0  1  1  1  1  1     1  1    0  0  1  1  1  1  1  1  1
 0  1  1  0  999  1  0  1  0  1  0  1     0  1    1  0  1  1  1  1  0  0  1
 0  0  1  1    0  1  1  1  1  1  1  0  …  0  0    0  0  1  1  1  1  0  1  1
 1  0  0  0    0  1  1  0  1  0  0  1     0  0    0  0  1  1  1  1  0  1  1
 1  0  0  0    0  1  0  0  0  1  0  0     0  0    1  0  1  1  0  1  0  1  0
 1  0  1  0    1  1  1  1  1  0  0  0     0  0    1  0  1  1  1  1  0  1  0
 1  0  0  1    1  1  0  1  1  1  0  0     0  1    1  0  1  0  1  1  0  0  1
 0  0  1  1    1  1  0  1  1  1  1  0  …  0  0    0  0  1  1  1  0  0  0  0
 1  0  1  0    0  0  1  0  1  0  1  0     0  1    0  0  1  1  1  1  0  0  0
 0  0  1  0    0  1  0  1  0  1  0  0     0  0    0  0  1  1  1 

In [3]:
#index for NA values
tmp_idx=findall(x->x==999,dataset)
dataset_origin[tmp_idx]

ndim = 2
niter = 30000
nburn = 5000
nthin = 5
nprint = 100

pr_mean_beta = 0.0
pr_mean_theta = 0.0
pr_mean_z = 0.0
pr_mean_w = 0.0

prior_a = 0.001
prior_b = 0.001

#pr_sd_beta = 10.0
#pr_sd_theta = 10.0

jump_beta = 0.25
jump_theta = 1.0
jump_z = 0.50
jump_w = 0.25

old_sigma_w = 1.0
old_sigma_z = 1.0

nsamp = size(dataset,1)
nitem = size(dataset,2)

count_item = sum(dataset,dims=1)
count_samp = sum(dataset,dims=2)

old_beta = randn(nitem)
old_theta = randn(nsamp)
old_z = randn(nsamp,ndim)
old_w = randn(nitem,ndim)

new_beta = zeros(nitem)
new_theta = zeros(nsamp)
new_z = zeros(nsamp,ndim)
new_w = zeros(nitem,ndim)

samp_like = zeros(nitem)
old_samp_dist = zeros(nitem)
new_samp_dist = zeros(nitem)

item_like = zeros(nsamp)
old_item_dist = zeros(nsamp)
new_item_dist = zeros(nsamp)

nmcmc = Int64((niter-nburn)/nthin)
sample_beta = zeros(nmcmc,nitem)
sample_theta = zeros(nmcmc,nsamp)
sample_z = zeros(nmcmc,nsamp,ndim)
sample_w = zeros(nmcmc,nitem,ndim)
sample_var_beta  = zeros(nmcmc)
sample_var_theta = zeros(nmcmc)
sample_mle = zeros(nmcmc)

accept_beta = zeros(nitem)
accept_theta = zeros(nsamp)
accept_z = zeros(nsamp)
accept_w = zeros(nitem)

distance = zeros(nsamp,nitem)


sample_impute = zeros(nmcmc,size(tmp_idx,1))

5000×150 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [4]:
for iter = 1:niter
    ##Imputation step============================================================================================================================================================================
    impute_idx=0
    for i = 1:nitem
        for k = 1:nsamp
            if dataset_origin[k,i] == 999 # if missing value
                impute_dist=0
                for j = 1:ndim
                    impute_dist += (old_z[k,j]-old_w[i,j])^2
                end                
                p_ki=1/(1+exp(-(old_beta[i]+old_theta[k]-impute_dist)))
                impute_value=rand(Binomial(1,p_ki))
                dataset[k,i]=impute_value
                
                if iter > nburn && iter % nthin == 0 #to check samples of imputed value
                    impute_idx=impute_idx+1
                    count = (Int64)((iter-nburn)/nthin)                
                    sample_impute[count,impute_idx]=impute_value
                end
            end
        end
    end
#     @printf "\n"
#     @printf "imputation done" 
#     @printf "\n"
    ##Estimation step============================================================================================================================================================================
    for i = 1:nitem
        for j = 1:ndim
            new_w[i,j] = rand(Normal(old_w[i,j],jump_w))
        end
        for k = 1:nsamp
            item_like[k] = old_item_dist[k] = new_item_dist[k] = 0.0
        end

        for k = 1:nsamp
            for j = 1:ndim
                old_item_dist[k] += (old_z[k,j]-old_w[i,j])^2
                new_item_dist[k] += (old_z[k,j]-new_w[i,j])^2
            end
            old_item_dist[k] = sqrt(old_item_dist[k])
            new_item_dist[k] = sqrt(new_item_dist[k])
            if  dataset[k,i] == 1
                item_like[k] -= -log(1.0 + exp(-(old_beta[i]+old_theta[k]-old_item_dist[k])))
                item_like[k] += -log(1.0 + exp(-(old_beta[i]+old_theta[k]-new_item_dist[k])))
            else
                item_like[k] -= -log(1.0 + exp(old_beta[i]+old_theta[k]-old_item_dist[k]))
                item_like[k] += -log(1.0 + exp(old_beta[i]+old_theta[k]-new_item_dist[k]))
            end
        end

        update_like_item = 0.0
        for k = 1:nsamp
            update_like_item += item_like[k]
        end
        num = den = 0.0
        for j = 1:ndim
            num += logpdf(Normal(pr_mean_w,old_sigma_w),new_w[i,j])
            den += logpdf(Normal(pr_mean_w,old_sigma_w),old_w[i,j])
        end
        ratio = update_like_item + (num - den)

        if ratio > 0.0
            accept = 1
        else
            un = rand()
            if log(un) < ratio
                accept = 1
            else
                accept = 0
            end
        end

        if accept == 1
            for j = 1:ndim
                old_w[i,j] = new_w[i,j]
            end
            accept_w[i] += 1.0 / niter
        else
            for j = 1:ndim
                new_w[i,j] = old_w[i,j]
            end
        end
    end

    for k = 1:nsamp
        for j = 1:ndim
            new_z[k,j] = rand(Normal(old_z[k,j],jump_z))
        end
        for i = 1:nitem
            samp_like[i] = old_samp_dist[i] = new_samp_dist[i] = 0.0
        end

        for i = 1:nitem
            for j = 1:ndim
                old_samp_dist[i] += (old_z[k,j]-old_w[i,j])^2
                new_samp_dist[i] += (new_z[k,j]-old_w[i,j])^2
            end
            old_samp_dist[i] = sqrt(old_samp_dist[i])
            new_samp_dist[i] = sqrt(new_samp_dist[i])
            if  dataset[k,i] == 1
                samp_like[i] -= -log(1.0 + exp(-(old_beta[i]+old_theta[k]-old_samp_dist[i])))
                samp_like[i] += -log(1.0 + exp(-(old_beta[i]+old_theta[k]-new_samp_dist[i])))
            else
                samp_like[i] -= -log(1.0 + exp(old_beta[i]+old_theta[k]-old_samp_dist[i]))
                samp_like[i] += -log(1.0 + exp(old_beta[i]+old_theta[k]-new_samp_dist[i]))
            end
        end

        update_like_samp = 0.0
        for i = 1:nitem
            update_like_samp += samp_like[i]
        end
        num = den = 0.0
        for j = 1:ndim
            num += logpdf(Normal(pr_mean_z,old_sigma_z),new_z[k,j])
            den += logpdf(Normal(pr_mean_z,old_sigma_z),old_z[k,j])
        end
        ratio = update_like_samp + (num - den)

        if ratio > 0.0
            accept = 1
        else
            un = rand()
            if log(un) < ratio
                accept = 1
            else
                accept = 0
            end
        end

        if accept == 1
            for j = 1:ndim
                old_z[k,j] = new_z[k,j]
            end
            accept_z[k] += 1.0 / niter
        else
            for j = 1:ndim
                new_z[k,j] = old_z[k,j]
            end
        end
    end

    mean_beta = mean(old_beta)
    post_a = prior_a + 0.5 * nitem
    post_b = prior_b
    for i = 1:nitem
        post_b += 0.5 * (old_beta[i]-mean_beta)^2
    end
    old_var_beta = rand(InverseGamma(post_a,post_b))
    pr_sd_beta = sqrt(old_var_beta)

    for i = 1:nitem
        new_beta[i] = rand(Normal(old_beta[i],jump_beta))
        old_like_beta = new_like_beta = 0.0
        for k = 1:nsamp
            beta_dist = 0.0
            for j = 1:ndim
                beta_dist += (old_z[k,j]-old_w[i,j])^2
            end
            beta_dist = sqrt(beta_dist)

            if dataset[k,i] == 1
                old_like_beta += -log(1.0 + exp(-(old_beta[i]+old_theta[k]-beta_dist)))
                new_like_beta += -log(1.0 + exp(-(new_beta[i]+old_theta[k]-beta_dist)))
            else
                old_like_beta += -log(1.0 + exp(old_beta[i]+old_theta[k]-beta_dist))
                new_like_beta += -log(1.0 + exp(new_beta[i]+old_theta[k]-beta_dist))
            end
        end

        num = logpdf(Normal(pr_mean_beta,pr_sd_beta),new_beta[i])
        den = logpdf(Normal(pr_mean_beta,pr_sd_beta),old_beta[i])
        ratio = (new_like_beta-old_like_beta) + (num-den)

        if ratio > 0.0
            accept = 1
        else
            un = rand()
            if log(un) < ratio
                accept = 1
            else
                accept = 0
            end
        end

        if accept == 1
            old_beta[i] = new_beta[i]
            accept_beta[i] += 1.0 / niter
        else
            new_beta[i] = old_beta[i]
        end
    end

    mean_theta = mean(old_theta)
    post_a = prior_a + 0.5 * nsamp
    post_b = prior_b
    for k = 1:nsamp
        post_b += 0.5 * (old_theta[k]-mean_theta)^2
    end
    old_var_theta = rand(InverseGamma(post_a,post_b))
    pr_sd_theta = sqrt(old_var_theta)

    for k = 1:nsamp
        new_theta[k] = rand(Normal(old_theta[k],jump_theta))
        old_like_theta = new_like_theta = 0.0
        for i = 1:nitem
            theta_dist = 0.0
            for j = 1:ndim
                theta_dist += (old_z[k,j]-old_w[i,j])^2
            end
            theta_dist = sqrt(theta_dist)

            if dataset[k,i] == 1
                old_like_theta += -log(1.0 + exp(-(old_beta[i]+old_theta[k]-theta_dist)))
                new_like_theta += -log(1.0 + exp(-(old_beta[i]+new_theta[k]-theta_dist)))
            else
                old_like_theta += -log(1.0 + exp(old_beta[i]+old_theta[k]-theta_dist))
                new_like_theta += -log(1.0 + exp(old_beta[i]+new_theta[k]-theta_dist))
            end
        end

        num = logpdf(Normal(pr_mean_theta,pr_sd_theta),new_theta[k])
        den = logpdf(Normal(pr_mean_theta,pr_sd_theta),old_theta[k])
        ratio = (new_like_theta-old_like_theta) + (num-den)

        if ratio > 0.0
            accept = 1
        else
            un = rand()
            if log(un) < ratio
                accept = 1
            else
                accept = 0
            end
        end

        if accept == 1
            old_theta[k] = new_theta[k]
            accept_theta[k] += 1.0 / niter
        else
            new_theta[k] = old_theta[k]
        end
    end

    if iter > nburn && iter % nthin == 0
        count = (Int64)((iter-nburn)/nthin)

        mle = 0.0
        for k = 1:nsamp
            for i = 1:nitem
                distance[k,i] = 0.0
                for j = 1:ndim
                    distance[k,i] += (old_z[k,j]-old_w[i,j])^2
                end
                distance[k,i] = sqrt(distance[k,i])
                if dataset[k,i] == 1
                    mle += -log(1.0 + exp(-(old_beta[i]+old_theta[k]-distance[k,i])))
                else
                    mle += -log(1.0 + exp(old_beta[i]+old_theta[k]-distance[k,i]))
                end
            end
        end
        for k = 1:nsamp
            mle += logpdf(Normal(pr_mean_theta,pr_sd_theta),old_theta[k])
        end
        for i = 1:nitem
            mle += logpdf(Normal(pr_mean_beta,pr_sd_beta),old_beta[i])
        end

        for k = 1:nsamp
            for j = 1:ndim
                sample_z[count,k,j] = old_z[k,j]
            end
        end

        for i = 1:nitem
            for j = 1:ndim
                sample_w[count,i,j] = old_w[i,j]
            end
        end

        for i = 1:nitem
            sample_beta[count,i] = old_beta[i]
        end

        for k = 1:nsamp
            sample_theta[count,k] = old_theta[k]
        end

        sample_var_beta[count] = old_var_beta
        sample_var_theta[count] = old_var_theta

        sample_mle[count] = mle
    end

    if iter % nprint == 0
        @printf "%5d\n" iter
        for i = 1:nitem
            @printf "% .3f " old_beta[i]
        end
        @printf "\n"
        for k = 1:nitem
            @printf "% .3f " old_theta[k]
        end
        @printf "\n"
        for j = 1:ndim
            @printf "% .3f " old_z[1,j]
        end
        for j = 1:ndim
            @printf "% .3f " old_w[1,j]
        end
        @printf "\n"
        @printf "% .3f " old_var_beta
        @printf "% .3f\n" old_var_theta
    end
end

  100
 0.904 -0.962  2.276  1.404  0.450  4.709  2.245  2.403  1.412  1.492  0.501  1.199  0.681  0.408  3.753 -0.609  2.586 -1.147  0.335 -1.021 -1.917  0.684  0.379  0.976  0.063  0.304  0.462  0.662  0.755 -0.778  0.326  0.432 -0.309 -0.727  0.620  1.481 -0.312 -0.618  0.277 -0.848  1.417  0.530  0.119  0.961  1.617  0.189  0.573 -0.109  2.385  1.078  0.391 -0.344  0.465  0.555 -0.440 -0.246  1.916  0.279  0.109 -1.137  0.509 -0.488 -0.489  0.339  5.512  3.461  0.388  0.375  0.536 -0.599 -0.551  0.124 -2.686  0.572  1.488 -1.777  1.070 -1.287  3.589  0.761  0.569  0.436 -0.372  1.987 -0.484  3.129 -0.983 -1.269  0.061 -0.384 -0.131  1.447  0.949  4.318  0.441  1.147 -0.962  4.002  1.535 -0.217  1.347 -0.779 -1.001  3.540 -3.220  0.438 -0.670  0.418  0.920  0.331 -1.739  4.371  4.970  3.531  4.400 -2.088  0.997  1.684 
 1.403  1.305  2.010  1.081  2.396  0.660 -0.553 -0.872  1.039 -0.777  0.175  0.483  0.239  0.157 -1.825  1.833  1.473  0.126  1.565 -1.422  0.704  1.307 -0.605  0.627

 3.287  0.801
  600
 1.265 -0.123  3.131  1.930  1.310  4.928  1.028  3.344  2.107  2.212  1.074  1.414  1.479  1.360  5.001  0.424  3.011 -0.749  0.896  0.316 -1.432  1.933  1.373  1.319  0.918  1.011  1.337  1.734  1.405 -0.195  1.009  1.100  0.928  0.502  1.302  1.191 -0.217  0.085  1.083 -0.241  1.946  1.810  1.100  2.023  2.593  1.135  1.135  0.582  3.469  2.092  0.615  0.374  1.083  1.484  0.266  0.592  1.872  0.927 -0.440 -0.587  0.670 -1.023 -1.024  0.643  5.681  4.942  1.294  1.001  1.481  0.193  0.106  0.215 -1.368  0.027  2.161 -1.075  2.707 -1.491  3.563  1.492  2.107  1.177  0.355  2.676  0.218  3.472 -0.659 -1.043  0.784  0.342  0.158  2.271  2.009  5.273  0.637  2.671  0.953  4.709  1.989  0.590  1.183 -0.304 -0.939  4.232 -0.894  1.297 -0.126  1.762  1.685  0.999 -1.626  5.086  6.244  4.524  6.024 -1.206  1.719  2.787 
 0.843  0.703  1.113  0.365  1.717  0.153  0.420 -0.399  1.090 -0.441 -0.543 -0.339 -0.276 -0.111 -1.726  1.496  0.669  0.500  0.768 -2.738 -0.090  0.714

 3.411  1.002
 1100
 1.244 -1.384  3.557  2.091  0.912  5.487  1.165  3.354  2.105  2.203  1.121  2.379  1.289  1.247  4.458  0.592  3.339 -0.625  0.790 -0.217 -1.149  1.838  0.822  1.418  0.826  1.293  1.184  2.805  1.372  0.003  0.470  1.155  0.358  0.381  0.984  1.371 -0.196 -0.012  0.871  0.132  2.046  1.223  1.260  1.560  2.587  1.304  1.368  0.325  3.436  2.595  0.664  0.244  1.040  1.041  0.008  0.517  2.206  0.848 -0.217 -0.354  0.962 -0.705 -0.467  0.559  5.877  4.631  1.571  1.012  1.428  0.240 -0.082  0.348 -1.094  0.199  2.093 -0.912  2.359 -2.078  3.601  1.769  1.374  0.962  0.489  2.458  0.234  3.761  0.107 -1.176  0.881  0.338  0.304  1.991  1.735  5.146  0.604  1.801 -0.074  4.411  2.043  0.653  0.896 -0.596 -0.620  4.589 -1.978  1.014 -0.224  1.278  1.948  1.116 -1.192  5.025  5.409  4.170  5.743 -1.119  1.716  3.168 
 0.689  0.235  0.835  0.712  1.067 -0.223 -1.292 -1.535  0.449 -1.314  0.193  0.104  0.100 -0.039 -0.443  1.830  0.738  0.009  0.678 -1.235  0.287  1.054

 2.733  0.768
 1600
 1.010  0.354  3.028  2.194  1.240  5.502  1.075  3.367  1.861  2.185  0.954  2.181  1.428  1.194  4.678  0.362  2.957 -0.707  0.876 -0.332 -1.842  1.658  0.885  1.508  0.797  1.158  1.226  1.659  1.380  0.107  0.493  1.268  1.440  0.564  1.340  1.485 -0.307  0.060  0.919 -0.196  1.891  1.309  0.686  1.768  2.633  1.125  1.379  0.252  4.150  2.228  0.941  0.194  1.179  1.484 -0.019  0.930  3.093  1.274 -0.182 -0.462  1.143 -0.540 -0.914  0.396  6.136  5.066  1.058  0.956  1.548  0.149  0.009  0.407 -1.174  0.303  2.075 -0.738  3.222 -1.462  3.664  1.330  1.994  1.067  0.207  2.495  0.228  4.425 -0.746 -0.236  0.711  0.370  0.095  1.912  1.731  4.562  0.492  2.555  0.147  4.945  2.264 -0.076  1.041  0.045 -0.972  4.543 -1.266  1.223  0.125  1.058  1.555  1.070 -1.907  5.048  5.829  4.236  5.174 -1.401  1.554  2.920 
 0.625  0.561  1.001  1.233  1.557 -0.377 -1.111 -0.538  0.749 -0.643 -0.438 -0.046 -0.039 -0.071 -1.942  1.746  1.159 -0.010  1.061 -1.114 -0.170  0.704

 2.618  1.089
 2100
 1.123 -0.343  3.381  2.033  0.874  4.771  1.189  3.195  2.173  2.239  0.938  1.521  1.605  1.162  4.475  0.374  3.182 -0.539  1.024 -0.287 -0.708  2.507  1.045  1.607  0.753  0.880  1.082  1.884  1.400 -0.215  0.509  1.236  0.271  0.343  1.770  1.093 -0.204  0.151  1.041 -0.326  2.160  1.347  0.969  1.473  2.650  1.012  1.416  0.548  3.056  1.849  0.446 -0.025  0.994  1.222  0.028  0.487  2.102  1.463 -0.233 -0.661  0.737 -0.316 -0.427  0.618  5.403  5.126  1.238  1.159  1.552  0.038  0.214  0.330 -1.484  0.199  2.093 -0.485  3.672 -1.857  4.175  1.306  2.270  0.949  0.082  2.617  0.211  3.789 -0.191 -0.809  0.632  0.527  0.087  2.000  1.655  5.141  0.576  2.526 -0.016  4.748  2.089  0.157  1.209 -0.060 -0.845  4.417 -1.437  1.428 -0.474  1.114  2.318  0.957  0.146  4.850  5.403  3.968  6.833 -1.494  1.943  3.362 
 1.043  0.787  1.036  1.189  0.384 -0.084 -0.929  0.284 -0.127 -1.118  0.035  0.335 -0.100  0.458 -2.036  1.265  0.596  0.325  0.435 -1.266  0.179  1.348

...

 3.073  0.862
27600
 1.239 -0.569  3.130  1.695  1.783  5.717  0.750  3.354  2.087  2.254  1.080  1.660  1.443  1.032  4.947  0.156  3.236  0.058  0.818 -0.352 -1.519  2.215  1.711  1.595  0.666  1.144  1.087  2.149  1.364 -0.196  0.484  1.451  0.479  0.325  1.711  1.111 -0.014  0.107  1.008 -0.099  2.117  1.287  0.956  1.805  3.149  0.876  1.435  0.499  3.156  1.644  1.181  0.744  1.113  1.388 -0.124  0.488  1.807  0.985 -0.267 -0.399  1.189 -0.446 -0.629  0.606  5.348  5.059  1.171  1.124  1.638  0.145  0.133  0.180 -0.595  0.216  2.143 -0.788  3.074 -1.869  3.307  1.562  1.371  1.081  0.286  2.439  0.079  3.731 -0.199 -0.614  0.932  0.498  0.214  1.874  1.730  4.721  0.647  2.627  0.490  4.845  2.178  0.458  1.102 -0.574 -0.699  4.515 -2.337  1.033 -0.295  1.469  1.635  1.029 -1.073  5.247  5.711  4.244  5.693 -1.112  1.955  2.981 
 0.982  0.410  1.228  0.731  0.651 -0.065  0.270 -0.762  0.710 -1.526 -0.569 -0.455  0.329 -0.086 -2.128  0.984  0.507  0.513  0.784 -1.827 -0.054  0.868

 3.091  0.725
28100
 1.126 -0.811  3.178  2.078  1.567  6.585  1.156  3.527  2.232  2.346  1.190  1.711  1.409  1.152  4.659  0.659  2.881 -0.990  1.032 -0.164 -1.523  1.803  1.250  1.523  0.863  0.984  1.379  1.745  1.923  0.058  0.780  1.340  0.395  0.872  2.201  1.369 -0.182  0.256  1.256 -0.085  2.044  1.608  1.065  1.849  2.275  0.812  1.193  0.417  3.314  2.121  0.890  0.429  1.103  1.504  0.048  0.573  1.738  0.920 -0.058 -0.607  0.860 -0.546 -0.703  0.646  6.002  5.661  1.291  1.237  1.482  0.085  0.378  0.202 -1.184  0.111  2.014 -0.980  2.570 -1.196  3.404  1.463  1.659  1.141  0.327  2.609 -0.074  4.732  0.192 -0.209  0.935  0.349  0.093  1.876  1.943  5.672  0.682  1.942  0.607  5.039  2.151  0.323  1.187 -0.131 -0.126  4.504 -1.908  1.125  0.270  1.288  1.811  0.957 -1.179  5.259  6.392  5.023  5.214 -1.335  1.766  2.635 
 1.151  0.087  1.099  0.300  1.327 -0.886 -1.251 -0.018  0.343 -1.046 -0.708  0.104  0.417 -1.209 -0.829  1.299  1.698 -0.078  0.961 -1.726 -0.304  0.875

28600
 1.104 -0.386  2.975  2.217  1.052  5.077  0.968  3.253  2.256  2.165  1.234  1.660  1.531  1.775  4.811  0.558  2.996 -0.381  0.948 -0.343 -1.090  1.641  1.179  1.380  0.647  1.797  1.513  2.755  1.477 -0.296  0.568  1.122  1.232  0.802  1.508  1.293  0.043  0.036  1.378 -0.288  2.162  1.445  1.776  2.239  2.532  0.886  1.188  0.660  2.901  1.789  1.252  0.015  0.965  1.476  0.010  0.472  1.792  0.756 -0.276 -0.322  0.693 -0.499 -0.448  0.460  6.073  5.279  0.906  1.063  1.387  0.339  0.024  0.366 -0.438  0.402  2.050 -0.976  3.128 -1.064  3.518  1.265  1.303  0.862  0.141  2.623  0.232  3.980 -0.149 -0.849  0.692  0.289  0.114  2.152  1.843  4.873  0.492  2.276  0.409  4.491  2.016  0.519  1.145 -0.221 -0.596  4.227 -1.679  1.253 -0.312  1.063  1.692  0.983 -1.266  5.719  5.720  4.225  5.232 -1.379  1.819  3.518 
 0.638  0.871  0.970  0.508  1.191 -0.183 -1.317 -0.914  0.554 -0.617  0.359  0.122 -0.942 -0.401 -1.766  0.870  0.583 -0.025  0.284 -1.961 -0.522  0.678  0.107  0.056

29100
 1.121 -0.478  3.098  2.223  1.414  5.442  1.106  3.319  2.061  2.123  1.137  1.384  1.297  1.698  4.721  0.210  2.836 -0.475  0.895 -0.198 -1.461  1.922  0.589  1.273  0.745  1.107  1.102  1.714  1.489  0.350  0.541  1.221  0.335  0.786  2.022  1.238 -0.136  0.079  0.611 -0.114  1.920  1.662  0.872  1.922  2.322  1.091  1.094  0.347  3.527  1.919  0.746  0.448  0.929  1.188 -0.101  0.469  1.650  0.670 -0.483 -0.452  0.787 -0.761 -0.585  0.457  5.531  5.164  1.045  0.988  1.447  0.299 -0.114  0.292 -1.238 -0.061  2.206 -0.965  3.312 -2.193  3.797  1.745  2.179  1.062  0.379  2.811  0.084  4.303 -0.829 -1.118  0.841  0.553  0.056  1.763  2.020  4.952  0.488  2.973  0.812  4.676  2.079  0.228  1.034  0.118 -0.909  5.364 -2.361  1.070  0.490  1.109  2.900  0.981 -0.815  5.595  5.721  3.993  5.884 -1.252  1.620  2.208 
 1.435  0.617  1.160  0.337  1.202 -0.349 -0.656 -0.919  0.187 -1.445 -0.112  0.058 -0.041 -0.970 -2.702  0.542  0.413 -0.340  0.771 -1.603 -0.196  0.884 -0.541  0.263

 2.727  0.970
29600
 1.034 -0.283  2.963  2.091  1.445  5.788  1.086  3.515  1.996  2.098  1.074  1.780  1.327  1.560  4.599  0.373  3.004  0.212  1.008 -0.241 -1.591  2.730  0.862  1.635  0.927  0.793  1.112  1.573  1.912  0.061  0.574  1.659  0.890  0.558  2.147  1.433  0.095 -0.086  0.707 -0.073  1.971  1.191  0.903  1.642  2.633  0.756  1.227  0.391  3.410  1.545  0.911  0.127  1.187  1.571  0.233  0.608  1.965  0.963 -0.256 -0.287  1.072 -0.548 -0.823  0.617  5.691  4.999  1.129  1.083  1.852  0.231  0.141  0.279 -1.042  0.206  2.353 -0.739  2.677 -1.058  3.742  1.393  2.414  1.300  0.310  2.572  0.173  4.700  0.079 -1.631  0.783  0.355  0.181  1.917  1.936  6.251  0.432  2.426  0.276  5.592  2.191  0.026  1.193  0.380 -0.632  4.766 -2.230  1.322 -0.029  1.488  2.306  0.906 -0.512  4.646  5.991  4.815  5.395 -1.294  1.546  2.472 
 0.554  0.599  1.450  0.450  1.271 -0.304 -0.976 -1.501  0.016 -1.368 -0.277 -0.063  0.056 -0.233 -2.341  1.111  0.778 -0.090  0.792 -1.460 -0.211  0.644

In [5]:
writedlm("result/impute.txt", sample_impute)
writedlm("result/beta.txt", sample_beta)
writedlm("result/theta.txt", sample_theta)
for j = 1:ndim
    oname = string("result/z",j,".txt")
    writedlm(oname, sample_z[:,:,j])
end
for j = 1:ndim
    oname = string("result/w",j,".txt")
    writedlm(oname, sample_w[:,:,j])
end
writedlm("result/var_beta.txt", sample_var_beta)
writedlm("result/var_theta.txt", sample_var_theta)
writedlm("result/mle.txt", sample_mle)
writedlm("result/impute.txt", sample_impute)
writedlm("result/accept_beta.txt", accept_beta)
writedlm("result/accept_theta.txt", accept_theta)
writedlm("result/accept_z.txt", accept_z)
writedlm("result/accept_w.txt", accept_w)